In [ ]:
import compyute as cp

In [ ]:
# import matplotlib.pyplot as plt
# from transformer_s import  PositionalEncoding

# pe = PositionalEncoding(256, 384, 1e2)

# plt.imshow(pe.encodings.squeeze(), cmap="grey")

In [ ]:
from compyute.preprocessing.text import CharacterTokenizer

with open("../data/tinyshakespeare.txt", "r") as f:
    data = f.read()

chars = sorted(list(set(data)))
tokenizer = CharacterTokenizer()
tokenizer.vocab = {i:c for i, c in enumerate(chars)}
tokenizer.ivocab = {c:i for i, c in enumerate(chars)}

In [ ]:
from transformer_s import get_causal_mask, Transformer

mask = get_causal_mask((256, 256))

model = Transformer(
    n_embeddings=tokenizer.vocab_size,
    embedding_dim=384,
    ffwd_channels=4*384,
    n_heads=6,
    n_blocks=6,
    max_seq_len=256,
    mask=mask
)

model.to_device(cp.cuda)
state_dict = cp.load("../transformer_shakespeare_test_10.cp")
model.load_state_dict(state_dict["model"])

In [ ]:
# from sklearn.manifold import TSNE
# import matplotlib.pyplot as plt

# # get numpy array of embedding table
# embs = model.token_emb.w.to_cpu()

# # reduce dimensions to 2 to make
# tsne = TSNE(random_state=0).fit_transform(embs.data)

# # plot results
# plt.figure(figsize=(8, 8))
# plt.scatter(x=tsne[:,0], y=tsne[:,1], s=100)
# for i in range(len(tsne)):
#     char = tokenizer.decode(cp.tensor([i]))
#     plt.text(x=tsne[i,0], y=tsne[i,1], s=char, ha="center", va="center", color="white")
# plt.grid(linestyle="dashed", linewidth=0.5)

In [ ]:
output = ""
context = "\n"
print(context, end="")

context = tokenizer.encode(context)  # encode context
context = cp.tensor(context, cp.cuda, cp.int32).to_shape((1, -1))  # insert batch dim

for _ in range(500):
    logits = model(context)[0, -1].to_cpu()  # get logits of last token
    probs = cp.nn.functional.softmax(logits)  # convert to probs
    topk_probs, topk_indices = cp.topk(probs, 50)  # get top 50 probs
    topk_probs /= topk_probs.sum()  # normalize probs
    index = cp.random.multinomial(x=50, p=topk_probs, shape=(1,))  # sample
    index = topk_indices[index]  # get token id
    char = tokenizer.decode(index.to_list())
    output += char

    print(char, end="")
    
    index = index.to_shape((1, 1))
    context = cp.append(context[:, 1:], values=index, axis=1).to_int()  # append to previous context

# with open("transformer_shakespeare.txt", "w") as f:
#     f.write(output)